In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
cp.fuse(np.float32)
h = 1.

In [ ]:
v= cp.array([-0.00606533, -0.02837768, -0.20481078, -0.05524456,  0.00408442, -0.02378791, -0.11289296, -0.09047946, -0.0828985,   0.01015773]) # Real life
v_0 = cp.array([-0.26251362, -0.22397083, -0.28459696, -0.14160629,  0.11507459,
       -0.01314795,  0.00368215, -0.2233519 , -0.0494188 , -0.09833207])
v_0 = cp.array([ 1.02602951,  1.59187996,  0.23337749, -0.62634188,  0.16052645,
       -0.47135837,  0.21609987,  0.45909724,  0.70707697, -0.76436048])
v_1 = cp.array([-0.09152057,  0.26501426, -0.26361748, -0.09584528,  0.07564258,
       -0.28932995, -0.15172387, -0.17311338, -0.02250007, -0.02662765])
v_2 = cp.array([ 0.02806491,  0.18164134, -0.02569311,  0.08406244,  0.09760685,
       -0.2754576 , -0.18692242,  0.05429335, -0.05959692, -0.16104073])
v_3 = cp.array([ 0.00719622, -0.01367537, -0.04378771,  0.15642576,  0.03295938,
       -0.1364489 , -0.02709714, -0.16822205, -0.15617831, -0.05832736])

v_array = [v_0, v_1, v_2, v_3]
exp_var = cp.sum(cp.square(v))
def expected_var(v):
    return cp.sum(cp.square(v))

In [ ]:
# Generating vectors to have a variance close to the real one v
eps = 7e-1
v_generated = v - cp.random.normal(0,eps,len(v))
print(cp.linalg.norm(expected_var(v_generated) - expected_var(v)))
v_generated

In [ ]:
# Variable & functions def: 
def f_vec(X,v):
    return v.T@X

def f_vec_squared(X,v):
    return (v.T@X)**2

d = 10 # lenght of random vectors

In [ ]:
def estim_var_MLMC(N,M):
    """
    Parameters:
    - N: Number of samples for each Y_ell (howw much we generate Y to compute one variance)
    - M: array with sample lengh of each level [M_0,...,M_L] (how much variances we generate)

    """
    # Variance at level 0
    X_M_ell = cp.random.standard_normal((N,d,M[0]))
    Y_ell_M_ell = f_vec(X_M_ell,v_0)
    Var_MLMC = cp.var(Y_ell_M_ell, axis=1, ddof=1)
    
    # Loop on M to have correction V^(ell)_{M_ell}(Y_ell) - V^(ell)_{M_ell}(Y_{ell-1})
    for ell in range(1,len(M)):
        X_M_ell = cp.random.standard_normal((N,d,M[ell]))
        Y_ell_M_ell = f_vec(X_M_ell,v_array[ell])
        Y_ell_M_ell_minus_1 = f_vec(X_M_ell,v_array[ell-1])
        Var_MLMC += cp.var(Y_ell_M_ell, axis=1, ddof=1) - cp.var(Y_ell_M_ell_minus_1, axis=1, ddof=1)

    return Var_MLMC

def estim_var_MLMC_log(N,M):
    """
    Parameters:
    - N: Number of samples for each Y_ell (howw much we generate Y to compute one variance)
    - M: array with sample lengh of each level [M_0,...,M_L] (how much variances we generate)

    """
    # Variance at level 0
    X_M_ell = cp.random.standard_normal((N,d,M[0]))
    Y_ell_M_ell = f_vec(X_M_ell,v_0)
    Var_log_MLMC = cp.log(cp.var(Y_ell_M_ell, axis=1, ddof=1))
    
    # Loop on M to have correction V^(ell)_{M_ell}(Y_ell) - V^(ell)_{M_ell}(Y_{ell-1})
    for ell in range(1,len(M)):
        X_M_ell = cp.random.standard_normal((N,d,M[ell]))
        Y_ell_M_ell = f_vec(X_M_ell,v_array[ell])
        Y_ell_M_ell_minus_1 = f_vec(X_M_ell,v_array[ell-1])
        Var_log_MLMC += np.log(cp.var(Y_ell_M_ell, axis=1, ddof=1)) - cp.log(cp.var(Y_ell_M_ell_minus_1, axis=1, ddof=1))

    return cp.exp(Var_log_MLMC)

In [ ]:
N = 1000000
M = [18]
X_M_ell = cp.random.standard_normal((N,d,M[0]))
Y_ell_M_ell_squared = f_vec_squared(X_M_ell,v)
Y_ell_M_ell = f_vec(X_M_ell,v)

print(cp.mean(Y_ell_M_ell))

print("Var en calculant avec cp.var (ddof = 1) =", cp.var(Y_ell_M_ell, axis=1, ddof=1))
print("Var en calculant avec cp.var + correction =", M[0]/(M[0]-1)*cp.var(Y_ell_M_ell, axis=1))
print("var en calculant à la main la somme =", cp.sum(Y_ell_M_ell_squared, axis=1)/(M[0]-1))

In [ ]:
def M_ell_tab(eta, V_ell, C_ell,m=0):
    """
    eta : budget
    V_ell : tableau contenant [V_0,V_1,...,V_L]
    C_ell : tableau contenant [C_0,C_1,...,C_L]

    return : la formule M_ell au dessus qui nous donne le nombre d'élement optimal à simuler
    """
    eta_tilde = eta # - m*cp.sum(...)
    S_L = (cp.sqrt(cp.multiply(V_ell,C_ell))).sum()

    return m + 1 + cp.floor(
        (eta_tilde/S_L)*cp.sqrt(V_ell*1/C_ell)
    )

In [ ]:
N_pilote = 10000
X = cp.random.standard_normal((d,N_pilote))
V = [cp.var(f_vec_squared(X,v_0))]
for i in range(1,len(v_array)):
    V.extend([cp.var(f_vec_squared(X,v_array[i]) - f_vec_squared(X,v_array[i-1]))])

V_ell = cp.array(V)
C_ell = cp.array([.25,0.5,0.75,1.])

In [ ]:
N=10000
budget_eta = np.logspace(1,4,10)

In [ ]:
bias_tab = []
var_MLMC_tab = []
bias_log_tab = []
var_MLMC_log_tab = []


for eta in tqdm(budget_eta):
    # Pour chaque budget eta, calcul de nos M_0 et M_1
    M = list(map(int,M_ell_tab(eta,V_ell,C_ell)))
    
    # Calcul de nos tableaux de nos estimateurs de taille N
    Tab_var_MLMC = estim_var_MLMC(N,M)
    var_MLMC_tab.append(cp.var(Tab_var_MLMC))
    bias_tab.append(cp.square(cp.mean(Tab_var_MLMC) - expected_var(v)))

    Tab_var_MLMC = None
    cp.get_default_memory_pool().free_all_blocks()

    Tab_var_MLMC_log = estim_var_MLMC_log(N,M)
    var_MLMC_log_tab.append(cp.var(Tab_var_MLMC_log))
    bias_log_tab.append(cp.square(cp.mean(Tab_var_MLMC_log) - expected_var(v)))

    Tab_var_MLMC_log = None
    cp.get_default_memory_pool().free_all_blocks()

var_MLMC_tab = [x.item() for x in var_MLMC_tab]
bias_tab = [x.item() for x in bias_tab]

var_MLMC_log_tab = [x.item() for x in var_MLMC_log_tab]
bias_log_tab = [x.item() for x in bias_log_tab]

In [ ]:
# Plot log-log
plt.figure(figsize=(10, 6))

# Plot des données
plt.loglog(budget_eta, bias_tab, marker='o', label='MSE MLMC')
plt.loglog(budget_eta, var_MLMC_tab, marker='s', label='Var MLMC')
plt.loglog(budget_eta, bias_log_tab, marker='^', label='MSE Log-MLMC')
plt.loglog(budget_eta, var_MLMC_log_tab, marker='x', label='Var Log-MLMC')


# Ajouter titre et légendes
plt.title(f'MSE et variance en fonction du budget pour n={n}')
plt.xlabel('Budget eta')
plt.ylabel('MSE/Variance')
plt.legend()

# Afficher le plot
plt.grid(True)
plt.show()

# MSE en fonction de n
budget fixé à $10^3$

In [ ]:
budget_eta_int = 1e3
samples_N = list(map(int,np.logspace(2,5,5)))

In [ ]:
bias_tab = []
var_MLMC_tab = []
bias_log_tab = []
var_MLMC_log_tab = []


for N in tqdm(samples_N):
    # Pour chaque budget eta, calcul de nos M_0 et M_1
    M = list(map(int,M_ell_tab(eta,V_ell,C_ell)))
    
    # Calcul de nos tableaux de nos estimateurs de taille N
    Tab_var_MLMC = estim_var_MLMC(N,M)
    var_MLMC_tab.append(cp.var(Tab_var_MLMC))
    bias_tab.append(cp.square(cp.mean(Tab_var_MLMC) - expected_var(v)))

    Tab_var_MLMC = None
    cp.get_default_memory_pool().free_all_blocks()

    Tab_var_MLMC_log = estim_var_MLMC_log(N,M)
    var_MLMC_log_tab.append(cp.var(Tab_var_MLMC_log))
    bias_log_tab.append(cp.square(cp.mean(Tab_var_MLMC_log) - expected_var(v)))

    Tab_var_MLMC_log = None
    cp.get_default_memory_pool().free_all_blocks()

var_MLMC_tab = [x.item() for x in var_MLMC_tab]
bias_tab = [x.item() for x in bias_tab]

var_MLMC_log_tab = [x.item() for x in var_MLMC_log_tab]
bias_log_tab = [x.item() for x in bias_log_tab]